https://github.com/reinforceio/tensorforce/blob/master/tensorforce/models/q_nstep_model.py
    
    has code using expand dims etc - check vs what im doing
    
whats is the correct approach for the initial states
- padding with same state
- never training until you have n steps of experience in the buffer

In [1]:
from collections import defaultdict, deque, namedtuple

from functools import partial

import random

import numpy as np

from energy_py.agents.memory import Memory, Experience


class HistoryMemory(Memory):
    def __init__(self, size, n_step, obs_shape, action_shape):
        
        super().__init__(size, obs_shape, action_shape)
        self.n_step = n_step
        
        self.buffer = defaultdict(partial(deque, maxlen=self.n_step))
    
        self.experiences = deque(maxlen=self.size)
        
    def __repr__(self):
        return '<{} n_step return memory size={}>'.format(self.n_step,
                                                          self.size)
    def __len__(self):
        return len(self.experiences)
    

    def remember(self, o, a, r, next_o, done):
        
        self.buffer['observation'].append(
            np.array(o).reshape(1, *self.shapes['observation']))
        
        self.buffer['action'].append(
            np.array(a).reshape(1, *self.shapes['action']))        
        
        self.buffer['reward'].append(
            np.array(r).reshape(1, *self.shapes['reward']))
        
        self.buffer['next_observation'].append(
            np.array(next_o).reshape(1, *self.shapes['next_observation']))
        
        self.buffer['done'].append(
            np.array(done).reshape(1, *self.shapes['done']))
        
        if len(self.buffer['observation']) >= self.n_step:
            
            experience_dims = []
            for field in Experience._fields:
                buffer = self.buffer[field]
                hist = np.array(buffer).reshape(1, self.n_step, *buffer[0].shape[1:])
                experience_dims.append(hist)  
                
            experience = Experience(*experience_dims)
            self.experiences.append(experience)
    
    def get_batch(self, batch_size):
        """
        """
        
        sample_size = min(batch_size, len(self))                                                                                                                 
        batch = random.sample(self.experiences, sample_size)                                                                                                                                                                                                                                  

        batch_dict = {}
        for field in Experience._fields:
            arr = np.array([getattr(e, field) for e in batch])
            batch_dict[field] = arr.reshape(batch_size, self.n_step, *self.shapes[field])
            
        return batch_dict

/Users/adam/anaconda3/envs/energy_py/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/adam/anaconda3/envs/energy_py/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
mem = HistoryMemory(10, n_step=5, obs_shape=(4,), action_shape=(1,))

import gym

e = gym.make('CartPole-v0')
o = e.reset()
for step in range(10):
    act = e.action_space.sample()
    next_o, r, done, _ = e.step(act)
    mem.remember(o, act, r, next_o, done)
    o = next_o

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
batch = mem.get_batch(3)

In [4]:
batch['observation'].shape

(3, 5, 4)

In [5]:
# to learn from n_step returns

train_states = batch['observation'][:, 0]
train_actions = batch['action'][:, 0]

experienced_rewards = np.squeeze(batch['reward'][:])

discounts = [0.9**s for s in range(experienced_rewards.shape[1])]

experienced_returns = np.sum(experienced_rewards * discounts, axis=1)

In [6]:
experienced_rewards.shape

(3, 5)

In [7]:
experienced_returns

array([4.0951, 4.0951, 4.0951])

In [8]:
next_states = batch['next_observation'][:, -1]
next_states.shape

next_state_discount = 0.9**(experienced_rewards.shape[1] + 1)

In [9]:
next_state_discount

0.531441